<a href="https://colab.research.google.com/github/anoopsanka/retinal_oct/blob/main/notebooks/streamlit_app_representation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [Run streamlit app from a Google Colab Notebook](https://discuss.streamlit.io/t/free-streamlit-dev-environment-through-colaboratory/2778/12)

> andfanilo
>Just discovered that npm is available on Colab :exploding_head: so you can use any node package in Colab, which means localtunnel 3 to expose your Streamlit app to the world (until the notebook or the localtunnel server get shut down at least XD)

>so you can run the following 4 Colab cells, and get the exposed URL at the end. :
```python
!pip install streamlit
!npm install localtunnel
!streamlit run app.py &>/dev/null&
!npx localtunnel --port 8501
```

In [ ]:
%%capture
!pip install streamlit
!npm install localtunnel

In [ ]:
%%capture
# needed to upgrade in order to use rotate....
!pip install -U tensorflow_addons

In [ ]:
%%capture
!git clone https://github.com/anoopsanka/retinal_oct

In [ ]:
%cd retinal_oct

/content/retinal_oct


Reset the execution environment after streamlit installation

In [ ]:
%%writefile img_aug.py

import streamlit as st
import pandas as pd
import tensorflow as tf
import numpy as np
import functools
import tensorflow_datasets as tfds
import tensorflow_addons as tfa

import os
import wandb
import numpy as np
from core.models.simclr_model import Pretrained_SimCLR_Model
from sklearn.decomposition import PCA
import plotly.express as px

# from importlib.util import find_spec
# if find_spec("retinal_oct/core") is None:
#     import sys
#     sys.path.append('..')
    
# from core.datasets import RetinaDataset




def train_classification_aug(img, lb, 
                            img_size=128, 
                            brightness_delta= 0.8,
                            contrast_delta  = 0.5,
                            saturation_delta= 0.5,
                            hue_delta       = 0.2,
                            max_rot_angle   = 45.):
  img = tf.cast(img, dtype=tf.float32)/255.
  IMG_SIZE = img_size

  angle_rad = max_rot_angle / 180. * np.pi

  padding = IMG_SIZE // 4
  precrop_shape = IMG_SIZE + padding

  img = tf.image.resize(img, (precrop_shape, precrop_shape))
  img = tf.image.random_crop(img, (IMG_SIZE, IMG_SIZE, 3))
  img = tf.image.random_brightness(img, brightness_delta)
  img = tf.image.random_contrast  (img, 1-contrast_delta,  1+contrast_delta)
  img = tf.image.random_saturation(img, 1-saturation_delta,1+saturation_delta)
  img = tf.image.random_hue       (img, hue_delta)

  img = tf.image.random_flip_left_right(img)
  img = tfa.image.rotate(img, (tf.random.uniform(shape=(1,)) - 0.5)*2 *angle_rad )

  img = tf.clip_by_value(img, 0., 1.0)
  return img, lb #tf.one_hot(lb, 4, )

##################################################################################

st.write("# Understand what happens during a augmentation!!")
st.sidebar.header('User Input Parameters')
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

# ds_train, ds_train_info = tfds.load('RetinaDataset', split='train', shuffle_files=True, as_supervised=True, with_info=True)
ds_test  = tfds.load('RetinaDataset', split='test', shuffle_files=True, as_supervised=True)

 
def user_input_augmentation():
  # rotation = st.sidebar.slider('Rotation', 0., 180., 10.)
  hue      = st.sidebar.slider('Hue', -1., 1., 0.1)
  brightness= st.sidebar.slider('Brightness', -1., 1., 0.1)
  contrast  = st.sidebar.slider('Contrast', -1., 1., 0.1)
  saturation  = st.sidebar.slider('Saturation', -1., 1., 0.1)
  rotation  = st.sidebar.slider('Rotation', -90., 90., 5.)

  data = {
    'hue': hue,
    'brightness': brightness,
    'contrast': contrast,
    'saturation': saturation,
    'rotation': rotation
  }
  features = pd.DataFrame(data, index=[0])
  return features
df = user_input_augmentation()
st.subheader('User Input parameters')
st.write(df)


def image_aug(img, features):
  # img = tf.image.rotation(img, features['rotation'])
  IMG_SIZE = 128
  img = img[tf.newaxis,...]

  padding = IMG_SIZE // 4
  precrop_shape = IMG_SIZE + padding

  angle_rad =  np.pi/ 180.

  img = tf.image.resize(img, (precrop_shape, precrop_shape))
  img = tf.image.random_crop(img, (1, IMG_SIZE, IMG_SIZE, 3))

  img = tf.image.adjust_brightness(img, features['brightness'].values[0])
  img = tf.image.adjust_contrast(img, features['contrast'].values[0])
  img = tf.image.adjust_hue(img, features['hue'].values[0])
  img = tf.image.adjust_saturation(img, features['saturation'].values[0])
  img = tfa.image.rotate(img, features['rotation'].values[0]*angle_rad )
  img = tf.clip_by_value(img, 0, 255)
  return img[0].numpy()/255.

nrows = 4
ncols = 4

x_train = ds_test.take(nrows*ncols)

img_list = []
for img, lb in x_train:

  img_list.append(image_aug(img, df))

  if len(img_list) == 4:
    st.image(img_list)
    img_list = []

# pulled the weights


api = wandb.Api()
run = api.run("hisunnytang/OCT-keras-SimCLR/1ipksk06")
weight_file = "weights.108-0.97.hdf5"
if not os.path.exists(weight_file):
  run.file().download()

# initialize the model
config = run.config

model_loadweights = Pretrained_SimCLR_Model()

input_shape_base   = (None, config["IMG_SIZE"], config["IMG_SIZE"], 3)
input_shape_simclr = (None, config["IMG_SIZE"], config["IMG_SIZE"], 6)
model_loadweights.base_model.build(input_shape_base)
model_loadweights.build(input_shape_simclr)
model_loadweights.summary()
model_loadweights.load_weights('weights.108-0.97.hdf5')



# get the PCAs

img_list = []
for img, lb in ds_test.take(968):
  img_list.append(image_aug(img, df))

img_list = np.array(img_list)

proj, logits = model_loadweights(img_list, training=False)
proj_normed = proj.numpy() / np.square(proj.numpy()).mean(axis=-1, keepdims=True)**0.5


model_pcas = PCA(n_components=3)
pcas = model_pcas.fit_transform(proj_normed)

# retrieve the training labels
labels_train = [i.numpy() for i in ds_test.take(968).map(lambda img, lb: lb).batch(128)]
labels_train_ = np.hstack(labels_train)

# binding this to a dataframe
df_pcas = pd.DataFrame( pcas, columns = ['pca1', 'pca2', 'pca3'] )
df_pcas['label'] = labels_train_.astype(object)


# import streamline as st
# election = px.data.election()
st.header('PCA in 3D')
fig = px.scatter_3d(df_pcas, 
                    x="pca1", 
                    y="pca2", 
                    z="pca3", 
                    color="label", 
                    opacity=0.5,)
st.write(fig)



Writing img_aug.py


In [ ]:
pwd

'/content/retinal_oct'

In [ ]:
!streamlit run img_aug.py &>/dev/null&
!npx localtunnel --port 8501

npx: installed 22 in 2.87s
your url is: https://ancient-quail-27.loca.lt
^C


In [ ]:
!pip install wandb

     |████████████████████████████████| 1.9MB 4.2MB/s 
     |████████████████████████████████| 133kB 36.7MB/s 
     |████████████████████████████████| 102kB 9.0MB/s 
     |████████████████████████████████| 102kB 9.5MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp36-none-any.whl size=6490 sha256=f4be73d57cdc739ebd00b0fb8b59701d1b285df7443a9edd7ab631fc8710556b
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
  Created wheel for watchdog: filename=watchdog-0.10.4-cp36-none-any.whl size=74842 sha256=0384c2d91d1237d2cd342f1f7566a170856a275fdaeede10a8e35f522e9fb221
  Stored in directory: /root/.cache/pip/wheels/9e/11/04/5160b8815b0cc7cf574bdc6d053e510169ec264c8791b4ec3a
  Created wheel for pathtools: filename=pathtools-0.1.2-cp36-none-any.whl size=8785 sha256=ab9c9966f131750bb9360b4e5d8f73dcc3abc380d2dd4ddccba3ddfbc3c0dac4
  Stored in directory: /root/.cache/pip/wheels/0b/04/79/c3b0c3a0266a3cb4376da31e5bfe8bba0c4

In [ ]:
import wandb
api = wandb.Api()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
api